In [ ]:
from pydrake.all import (Box, DiagramBuilder,
                         HPolyhedron, LinearSystem,
                         LogVectorOutput,
                         Sphere,
                         StartMeshcat, 
                         MeshcatVisualizerCpp,SceneGraph, Simulator, StartMeshcat,
                         AddMultibodyPlantSceneGraph, Parser, CoulombFriction, Cylinder, RigidTransform,
                         SpatialInertia, Sphere, UnitInertia, MakeRenderEngineVtk, RenderEngineVtkParams,
                         MeshcatVisualizerParams)

from pydrake import geometry

#import_ipynb
from utils import getMinMaxBox

import matplotlib.pyplot as plt
import numpy as np

import pydrake.symbolic as symbolic

NameError: name 'import_ipynb' is not defined

### RRT (2D)

In [ ]:
class RRT:
 
    class Node:
        def __init__(self, p):
            self.p = np.array(p)
            self.parent = None

    def __init__(self, start, goal, obstacle_list, bounds, 
                 max_extend_length=3.0, path_resolution=0.5, 
                 goal_sample_rate=0.05, max_iter=100):
        self.start = self.Node(start)
        self.goal = self.Node(goal)
        self.bounds = bounds
        self.max_extend_length = max_extend_length
        self.path_resolution = path_resolution
        self.goal_sample_rate = goal_sample_rate
        self.max_iter = max_iter
        self.obstacle_list = obstacle_list
        self.node_list = []

    def plan(self):
        """Plans the path from start to goal while avoiding obstacles"""
        self.node_list = [self.start]
        for i in range(self.max_iter):
            # modify here: 
            # 1) Create a random node (rnd_node) inside 
            # the bounded environment
            # 2) Find nearest node (nearest_node)
            # 3) Get new node (new_node) by connecting
            # rnd_node and nearest_node. Hint: steer
            # 4) If the path between new_node and the
            # nearest node is not in collision, add it to the node_list
            rnd_node = self.get_random_node()
            nearest_node = self.get_nearest_node(self.node_list, rnd_node)

            new_node = self.steer(nearest_node, rnd_node, self.max_extend_length)

            if not self.collision(nearest_node, new_node, self.obstacle_list):
                self.node_list.append(new_node)
    
            # Don't need to modify beyond here
            # If the new_node is very close to the goal, connect it
            # directly to the goal and return the final path
            if self.dist_to_goal(self.node_list[-1].p) <= self.max_extend_length:
                final_node = self.steer(self.node_list[-1], self.goal, self.max_extend_length)
                if not self.collision(final_node, self.node_list[-1], self.obstacle_list):
                    return self.final_path(len(self.node_list) - 1)
        return None  # cannot find path

    def steer(self, from_node, to_node, max_extend_length=np.inf):
        """Connects from_node to a new_node in the direction of to_node
        with maximum distance max_extend_length
        """
        new_node = self.Node(to_node.p)
        d = from_node.p - to_node.p
        dist = np.linalg.norm(d)
        if dist > max_extend_length:
            # rescale the path to the maximum extend_length
            new_node.p  = from_node.p - d / dist * max_extend_length
        new_node.parent = from_node
        return new_node

    def dist_to_goal(self, p):
        """Distance from p to goal"""
        return np.linalg.norm(p - self.goal.p)

    def get_random_node(self):
        """Sample random node inside bounds or sample goal point"""
        if np.random.rand() > self.goal_sample_rate:
            # Sample random point inside boundaries
            rnd = self.Node(np.random.rand(2)*(self.bounds[1]-self.bounds[0]) + self.bounds[0])
        else:  
            # Select goal point
            rnd = self.Node(self.goal.p)
        return rnd
    
    @staticmethod
    def get_nearest_node(node_list, node):
        """Find the nearest node in node_list to node"""
        dlist = [np.sum(np.square((node.p - n.p))) for n in node_list]
        minind = dlist.index(min(dlist))
        return node_list[minind]
    
    @staticmethod
    def collision(node1, node2, obstacle_list):
        """Check whether the path connecting node1 and node2 
        is in collision with anyting from the obstacle_list
        """
        p1 = node2.p
        p2 = node1.p 
        for o in obstacle_list:
            center_circle = o[0:2]
            radius = o[2]
            d12 = p2 - p1 # the directional vector from p1 to p2
            # defines the line v(t) := p1 + d12*t going through p1=v(0) and p2=v(1)
            d1c = center_circle - p1 # the directional vector from circle to p1
            # t is where the line v(t) and the circle are closest
            # Do not divide by zero if node1.p and node2.p are the same.
            # In that case this will still check for collisions with circles
            t = d12.dot(d1c) / (d12.dot(d12) + 1e-7)
            t = max(0, min(t, 1)) # Our line segment is bounded 0<=t<=1
            d = p1 + d12*t # The point where the line segment and circle are closest
            is_collide = np.sum(np.square(center_circle-d)) < radius**2
            if is_collide:
                return True # is in collision
        return False # is not in collision
    
    def final_path(self, goal_ind):
        """Compute the final path from the goal node to the start node"""
        path = [self.goal.p]
        node = self.node_list[goal_ind]
        # modify here: Generate the final path from the goal node to the start node.
        # We will check that path[0] == goal and path[-1] == start
        while node is not None:
            path.append(node.p)
            node = node.parent

        return path

    def draw_graph(self):
        for node in self.node_list:
            if node.parent:
                plt.plot([node.p[0], node.parent.p[0]], [node.p[1], node.parent.p[1]], "-g")

NameError: name 'np' is not defined

In [ ]:
class RRTStar(RRT):
    
    class Node(RRT.Node):
        def __init__(self, p):
            super().__init__(p)
            self.cost = 0.0

    def __init__(self, start, goal, obstacle_list, bounds,
                 max_extend_length=5.0,
                 path_resolution=0.5,
                 goal_sample_rate=0.0,
                 max_iter=200,
                 connect_circle_dist=50.0
                 ):
        super().__init__(start, goal, obstacle_list, bounds, max_extend_length,
                         path_resolution, goal_sample_rate, max_iter)
        self.connect_circle_dist = connect_circle_dist
        self.goal = self.Node(goal)

    def plan(self):
        """Plans the path from start to goal while avoiding obstacles"""
        self.node_list = [self.start]
        for i in range(self.max_iter):
            # Create a random node inside the bounded environment
            rnd = self.get_random_node()
            # Find nearest node
            nearest_node = self.get_nearest_node(self.node_list, rnd)
            # Get new node by connecting rnd_node and nearest_node
            new_node = self.steer(nearest_node, rnd, self.max_extend_length)
            # If path between new_node and nearest node is not in collision:
            if not self.collision(new_node, nearest_node, self.obstacle_list):
                near_inds = self.near_nodes_inds(new_node)
                # Connect the new node to the best parent in near_inds
                new_node = self.choose_parent(new_node, near_inds)
                self.node_list.append(new_node)
                # Rewire the nodes in the proximity of new_node if it improves their costs
                self.rewire(new_node, near_inds)
        last_index, min_cost = self.best_goal_node_index()
        if last_index:
            return self.final_path(last_index), min_cost
        return None, min_cost

    def choose_parent(self, new_node, near_inds):
        """Set new_node.parent to the lowest resulting cost parent in near_inds and
        new_node.cost to the corresponding minimal cost
        """
        min_cost = np.inf
        best_near_node = None
        # modify here: Go through all near nodes and evaluate them as potential parent nodes by
        # 1) checking whether a connection would result in a collision,
        # 2) evaluating the cost of the new_node if it had that near node as a parent,
        # 3) picking the parent resulting in the lowest cost and updating
        #    the cost of the new_node to the minimum cost.
        for ind in near_inds:
            potential_parent = self.node_list[ind]
            if not self.collision(potential_parent, new_node, self.obstacle_list):
                potential_cost = self.new_cost(potential_parent, new_node)
                if potential_cost < min_cost:
                    min_cost = potential_cost
                    best_near_node = potential_parent
                
        
        # Don't need to modify beyond here
        new_node.cost = min_cost
        new_node.parent = best_near_node
        return new_node
    
    def rewire(self, new_node, near_inds):
        """Rewire near nodes to new_node if this will result in a lower cost"""
        # modify here: Go through all near nodes and check whether rewiring them
        # to the new_node would: 
        # A) Not cause a collision and
        # B) reduce their own cost.
        # If A and B are true, update the cost and parent properties of the node.
        for ind in near_inds: 
            current_node = self.node_list[ind]
            new_cost = self.new_cost(new_node, current_node)
            if not self.collision(current_node, new_node, self.obstacle_list) and new_cost < current_node.cost:
                current_node.cost = new_cost
                current_node.parent = new_node

        # Don't need to modify beyond here
        self.propagate_cost_to_leaves(new_node)

    def best_goal_node_index(self):
        """Find the lowest cost node to the goal"""
        min_cost = np.inf
        best_goal_node_idx = None
        for i in range(len(self.node_list)):
            node = self.node_list[i]
            # Has to be in close proximity to the goal
            if self.dist_to_goal(node.p) <= self.max_extend_length:
                # Connection between node and goal needs to be collision free
                if not self.collision(self.goal, node, self.obstacle_list):
                    # The final path length
                    cost = node.cost + self.dist_to_goal(node.p) 
                    if node.cost + self.dist_to_goal(node.p) < min_cost:
                        # Found better goal node!
                        min_cost = cost
                        best_goal_node_idx = i
        return best_goal_node_idx, min_cost

    def near_nodes_inds(self, new_node):
        """Find the nodes in close proximity to new_node"""
        nnode = len(self.node_list) + 1
        r = self.connect_circle_dist * np.sqrt((np.log(nnode) / nnode))
        dlist = [np.sum(np.square((node.p - new_node.p))) for node in self.node_list]
        near_inds = [dlist.index(i) for i in dlist if i <= r ** 2]
        return near_inds

    def new_cost(self, from_node, to_node):
        """to_node's new cost if from_node were the parent"""
        d = np.linalg.norm(from_node.p - to_node.p)
        return from_node.cost + d

    def propagate_cost_to_leaves(self, parent_node):
        """Recursively update the cost of the nodes"""
        for node in self.node_list:
            if node.parent == parent_node:
                node.cost = self.new_cost(parent_node, node)
                self.propagate_cost_to_leaves(node)

In [ ]:
def even_space_2d(path):
    reversed_path = list(reversed(path))
    new_path = []
    total_step = 0.1

    for i in range(len(reversed_path)-1):
        x1, y1 = reversed_path[i]
        x2, y2 = reversed_path[i+1]

        slope = (y2-y1)/(x2-x1)

        x_step = np.abs(total_step/np.sqrt(slope**2+1))
        y_step = np.abs(slope*x_step)

        if x1 < x2:
            x_factor = 1
        else:
            x_factor = -1

        if y1 < y2:
            y_factor = 1
        else:
            y_factor = -1

        path_x = np.arange(x1, x2, x_factor*x_step)
        path_y = np.arange(y1, y2, y_factor*y_step)

        zipped_path = list(zip(path_x, path_y))
        new_path += zipped_path

    return new_path

### RRT (3D)

In [ ]:
class RRT3D:
 
    class Node:
        def __init__(self, p):
            self.p = np.array(p)
            self.parent = None

    def __init__(self, start, goal, obstacle_list, bounds, 
                 max_extend_length=3.0, path_resolution=0.5, 
                 goal_sample_rate=0.05, max_iter=100):
        self.start = self.Node(start)
        self.goal = self.Node(goal)
        self.bounds = bounds #either (,2) or (3x2)
        self.max_extend_length = max_extend_length
        self.path_resolution = path_resolution
        self.goal_sample_rate = goal_sample_rate
        self.max_iter = max_iter
        self.obstacle_list = obstacle_list
        self.node_list = []

    def plan(self):
        """Plans the path from start to goal while avoiding obstacles"""
        self.node_list = [self.start]
        for i in range(self.max_iter):
            # modify here: 
            # 1) Create a random node (rnd_node) inside 
            # the bounded environment
            # 2) Find nearest node (nearest_node)
            # 3) Get new node (new_node) by connecting
            # rnd_node and nearest_node. Hint: steer
            # 4) If the path between new_node and the
            # nearest node is not in collision, add it to the node_list
            rnd_node = self.get_random_node()
            nearest_node = self.get_nearest_node(self.node_list, rnd_node)

            new_node = self.steer(nearest_node, rnd_node, self.max_extend_length)

            if not self.collision(nearest_node, new_node, self.obstacle_list):
                self.node_list.append(new_node)
    
            # Don't need to modify beyond here
            # If the new_node is very close to the goal, connect it
            # directly to the goal and return the final path
            if self.dist_to_goal(self.node_list[-1].p) <= self.max_extend_length:
                final_node = self.steer(self.node_list[-1], self.goal, self.max_extend_length)
                if not self.collision(final_node, self.node_list[-1], self.obstacle_list):
                    return self.final_path(len(self.node_list) - 1)
        return None  # cannot find path

    def steer(self, from_node, to_node, max_extend_length=np.inf):
        """Connects from_node to a new_node in the direction of to_node
        with maximum distance max_extend_length
        """
        new_node = self.Node(to_node.p)
        d = from_node.p - to_node.p
        dist = np.linalg.norm(d)
        if dist > max_extend_length:
            # rescale the path to the maximum extend_length
            new_node.p  = from_node.p - d / dist * max_extend_length
        new_node.parent = from_node
        return new_node

    def dist_to_goal(self, p):
        """Distance from p to goal"""
        return np.linalg.norm(p - self.goal.p)

    def get_random_node(self):
        """Sample random node inside bounds or sample goal point"""
        if np.random.rand() > self.goal_sample_rate:
            # Sample random point inside boundaries
            #rnd = self.Node(np.random.rand(3)*(self.bounds[1]-self.bounds[0]) + self.bounds[0])
            if len(self.bounds) == 2: #same bounds for all 
                rnd = self.Node(np.random.rand(3)*(self.bounds[1]-self.bounds[0]) + self.bounds[0])
            else:
                rnd_x = np.random.rand(1)*(self.bounds[0][1]-self.bounds[0][0]) + self.bounds[0][0]
                rnd_y = np.random.rand(1)*(self.bounds[1][1]-self.bounds[1][0]) + self.bounds[1][0]
                rnd_z = np.random.rand(1)*(self.bounds[2][1]-self.bounds[2][0]) + self.bounds[2][0]
                rnd = self.Node(np.concatenate([rnd_x,rnd_y,rnd_z]))
        else:  
            # Select goal point
            rnd = self.Node(self.goal.p)
        return rnd
    
    @staticmethod
    def get_nearest_node(node_list, node):
        """Find the nearest node in node_list to node"""
        dlist = [np.sum(np.square((node.p - n.p))) for n in node_list]
        minind = dlist.index(min(dlist))
        return node_list[minind]
    
    @staticmethod
    def collision(node1, node2, obstacle_list):
        """Check whether the path connecting node1 and node2 
        is in collision with anyting from the obstacle_list
        """
        p1 = node2.p
        p2 = node1.p 
        for o in obstacle_list:
            in_collision = True
            if len(o) == 4: # spherical obstacle
                in_collision = sphere_collision(p1, p2, o)
            elif len(o) == 6: # rectangular obstacle
                in_collision = box_collision(p1, p2, o)
            else:
                raise Exception("Unknown obstacle parameters. Must be spherical ([x,y,z,r]) or rectangular ([x,y,z,w,h,d]).")

            if in_collision:
                return True

        return False # is not in collision
    
    def final_path(self, goal_ind):
        """Compute the final path from the goal node to the start node"""
        path = [self.goal.p]
        node = self.node_list[goal_ind]
        # modify here: Generate the final path from the goal node to the start node.
        # We will check that path[0] == goal and path[-1] == start
        while node is not None:
            path.append(node.p)
            node = node.parent

        return path

    def draw_graph(self):
        for node in self.node_list:
            if node.parent:
                plt.plot([node.p[0], node.parent.p[0]], [node.p[1], node.parent.p[1]], [node.p[2], node.parent.p[2]], "-g")

In [ ]:
class RRTStar3D(RRT3D):
    
    class Node(RRT.Node):
        def __init__(self, p):
            super().__init__(p)
            self.cost = 0.0

    def __init__(self, start, goal, obstacle_list, bounds,
                 max_extend_length=5.0,
                 path_resolution=0.5,
                 goal_sample_rate=0.0,
                 max_iter=200,
                 connect_circle_dist=50.0
                 ):
        super().__init__(start, goal, obstacle_list, bounds, max_extend_length,
                         path_resolution, goal_sample_rate, max_iter)
        self.connect_circle_dist = connect_circle_dist
        self.goal = self.Node(goal)

    def plan(self):
        """Plans the path from start to goal while avoiding obstacles"""
        self.node_list = [self.start]
        for i in range(self.max_iter):
            # Create a random node inside the bounded environment
            rnd = self.get_random_node()
            # Find nearest node
            nearest_node = self.get_nearest_node(self.node_list, rnd)
            # Get new node by connecting rnd_node and nearest_node
            new_node = self.steer(nearest_node, rnd, self.max_extend_length)
            # If path between new_node and nearest node is not in collision:
            if not self.collision(new_node, nearest_node, self.obstacle_list):
                near_inds = self.near_nodes_inds(new_node)
                # Connect the new node to the best parent in near_inds
                new_node = self.choose_parent(new_node, near_inds)
                self.node_list.append(new_node)
                # Rewire the nodes in the proximity of new_node if it improves their costs
                self.rewire(new_node, near_inds)
        last_index, min_cost = self.best_goal_node_index()
        if last_index:
            return self.final_path(last_index), min_cost
        return None, min_cost

    def choose_parent(self, new_node, near_inds):
        """Set new_node.parent to the lowest resulting cost parent in near_inds and
        new_node.cost to the corresponding minimal cost
        """
        min_cost = np.inf
        best_near_node = None
        # modify here: Go through all near nodes and evaluate them as potential parent nodes by
        # 1) checking whether a connection would result in a collision,
        # 2) evaluating the cost of the new_node if it had that near node as a parent,
        # 3) picking the parent resulting in the lowest cost and updating
        #    the cost of the new_node to the minimum cost.
        for ind in near_inds:
            potential_parent = self.node_list[ind]
            if not self.collision(potential_parent, new_node, self.obstacle_list):
                potential_cost = self.new_cost(potential_parent, new_node)
                if potential_cost < min_cost:
                    min_cost = potential_cost
                    best_near_node = potential_parent
                
        # Don't need to modify beyond here
        new_node.cost = min_cost
        new_node.parent = best_near_node
        return new_node
    
    def rewire(self, new_node, near_inds):
        """Rewire near nodes to new_node if this will result in a lower cost"""
        # modify here: Go through all near nodes and check whether rewiring them
        # to the new_node would: 
        # A) Not cause a collision and
        # B) reduce their own cost.
        # If A and B are true, update the cost and parent properties of the node.
        for ind in near_inds: 
            current_node = self.node_list[ind]
            new_cost = self.new_cost(new_node, current_node)
            if not self.collision(current_node, new_node, self.obstacle_list) and new_cost < current_node.cost:
                current_node.cost = new_cost
                current_node.parent = new_node

        # Don't need to modify beyond here
        self.propagate_cost_to_leaves(new_node)

    def best_goal_node_index(self):
        """Find the lowest cost node to the goal"""
        min_cost = np.inf
        best_goal_node_idx = None
        for i in range(len(self.node_list)):
            node = self.node_list[i]
            # Has to be in close proximity to the goal
            if self.dist_to_goal(node.p) <= self.max_extend_length:
                # Connection between node and goal needs to be collision free
                if not self.collision(self.goal, node, self.obstacle_list):
                    # The final path length
                    cost = node.cost + self.dist_to_goal(node.p) 
                    if node.cost + self.dist_to_goal(node.p) < min_cost:
                        # Found better goal node!
                        min_cost = cost
                        best_goal_node_idx = i
        return best_goal_node_idx, min_cost

    def near_nodes_inds(self, new_node):
        """Find the nodes in close proximity to new_node"""
        nnode = len(self.node_list) + 1
        r = self.connect_circle_dist * np.sqrt((np.log(nnode) / nnode))
        dlist = [np.sum(np.square((node.p - new_node.p))) for node in self.node_list]
        near_inds = [dlist.index(i) for i in dlist if i <= r ** 2]
        return near_inds

    def new_cost(self, from_node, to_node):
        """to_node's new cost if from_node were the parent"""
        d = np.linalg.norm(from_node.p - to_node.p)
        return from_node.cost + d

    def propagate_cost_to_leaves(self, parent_node):
        """Recursively update the cost of the nodes"""
        for node in self.node_list:
            if node.parent == parent_node:
                node.cost = self.new_cost(parent_node, node)
                self.propagate_cost_to_leaves(node)

In [ ]:
def even_space_3d(path, d):

    reversed_path = list(reversed(path))
    new_path = []
    step_size = d

    for i in range(len(reversed_path)-1):
        x1, y1, z1 = reversed_path[i]
        x2, y2, z2 = reversed_path[i+1]

        d = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2) + 10**-6 #total length of segment

        #slope = (y2-y1)/(x2-x1)

        x_step = step_size*(x2-x1)/d
        y_step = step_size*(y2-y1)/d
        z_step = step_size*(z2-z1)/d

        if x_step == 0:
            x_step = 10**-6
        if y_step == 0:
            y_step = 10**-6
        if z_step == 0:
            z_step = 10**-6

        path_x = np.arange(x1, x2, x_step)
        path_y = np.arange(y1, y2, y_step)
        path_z = np.arange(z1, z2, z_step)

        max_len = max(len(path_x), len(path_y), len(path_z))

        if len(path_x) == 0:
            assert (x1 == x2)
            path_x = x1*np.ones((max_len,))
        if len(path_y) == 0:
            assert (y1 == y2)
            path_y = y1*np.ones((max_len,))
        if len(path_z) == 0:
            assert (z1 == z2)
            path_z = z1*np.ones((max_len,))

        zipped_path = list(zip(path_x, path_y, path_z))
        new_path += zipped_path

    return new_path
    

### Collision Function

In [ ]:
def sphere_collision(p1, p2, o):
    center_circle = o[0:3]
    radius = o[3]
    d12 = p2 - p1 # the directional vector from p1 to p2
    # defines the line v(t) := p1 + d12*t going through p1=v(0) and p2=v(1)
    d1c = center_circle - p1 # the directional vector from circle to p1
    # t is where the line v(t) and the circle are closest
    # Do not divide by zero if node1.p and node2.p are the same.
    # In that case this will still check for collisions with circles
    t = d12.dot(d1c) / (d12.dot(d12) + 1e-7)
    t = max(0, min(t, 1)) # Our line segment is bounded 0<=t<=1
    d = p1 + d12*t # The point where the line segment and circle are closest
    is_collide = np.sum(np.square(center_circle-d)) < radius**2

    return is_collide

In [ ]:
def box_collision(p1, p2, box):
    (b1, b2) = getMinMaxBox(box) #returns list of [xmin, ymin, zmin] and [xmax, ymax, zmax]
    return CheckLineBox(b1, b2, p1, p2)
    

In [ ]:
def getIntersectionAndInBox(fDst1, fDst2, P1, P2, B1, B2, Axis):
    if ( (fDst1 * fDst2) >= 0.0):
        #print('Case 9')
        return False
    if ( fDst1 == fDst2):
        #print('Case 10')
        return False

    Hit = P1 + (P2-P1) * ( -fDst1/(fDst2-fDst1) )
    #print(f'Hit: {Hit}')

    if ( Axis==1 and Hit[2] >= B1[2] and Hit[2] <= B2[2] and Hit[1] >= B1[1] and Hit[1] <= B2[1]):
        #print('Case 11')
        return True
    if ( Axis==2 and Hit[2] >= B1[2] and Hit[2] <= B2[2] and Hit[0] >= B1[0] and Hit[0] <= B2[0]):
        #print('Case 12')
        return True
    if ( Axis==3 and Hit[0] >= B1[0] and Hit[0] <= B2[0] and Hit[1] > B1[1] and Hit[1] <= B2[1]):
        #print('Case 13')
        return True

    #print('Case 14')
    return False


# returns true if line (L1, L2) intersects with the box (B1, B2)
# returns intersection point in Hit
# Returns False if on boundary
def CheckLineBox(B1, B2, L1, L2):
    if (L2[0] < B1[0] and L1[0] < B1[0]):
        #print('Case 1: X value outside min value')
        return False
    if (L2[0] > B2[0] and L1[0] > B2[0]):
        #print('Case 2: X value outside max value')
        return False
    if (L2[1] < B1[1] and L1[1] < B1[1]):
        #print('Case 3: Y value outside min value')
        return False
    if (L2[1] > B2[1] and L1[1] > B2[1]):
        #print('Case 4: Y value outside max value')
        return False
    if (L2[2] < B1[2] and L1[2] < B1[2]):
        #print('Case 5: Z value outside min value')
        return False
    if (L2[2] > B2[2] and L1[2] > B2[2]):
        #print('Case 6: Z value outside max value')
        return False
   
    if (L1[0] >= B1[0] and L1[0] <= B2[0] and L1[1] >= B1[1] and L1[1] <= B2[1] and L1[2] >= B1[2] and L1[2] <= B2[2]):
        #print('Case 7: Completely w/in')
        return True

    # L1[0]-B1[0]: dist of first point to min x
    # L2[0]-B1[0]: dist of 2nd point to min x
    if getIntersectionAndInBox(L1[0]-B1[0], L2[0]-B1[0], L1, L2, B1, B2, 1) or \
    getIntersectionAndInBox(L1[1]-B1[1], L2[1]-B1[1], L1, L2, B1, B2, 2) or \
    getIntersectionAndInBox(L1[2]-B1[2], L2[2]-B1[2], L1, L2, B1, B2, 3) or \
    getIntersectionAndInBox(L1[0]-B2[0], L2[0]-B2[0], L1, L2, B1, B2, 1) or \
    getIntersectionAndInBox(L1[1]-B2[1], L2[1]-B2[1], L1, L2, B1, B2, 2) or \
    getIntersectionAndInBox(L1[[0]]-B2[2], L2[2]-B2[2], L1, L2, B1, B2, 3):
        #print('Case 8')
        return True
    
    #print('No cases')
    return False

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=931e8205-9a5c-4b8e-a3da-167564211c03' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>